In [1]:
from src.models import Exponential_Model
from src.criterion import right_censored,RightCensorWrapper,RankingWrapper
from src.load_data import load_datasets,load_dataframe
from src.utils import train_robust,lower_bound
from src.visualizations import visualize_population_curves_attacked,visualize_individual_curves_attacked,visualize_individual_curves_changes
from src.metrics import concordance

from torch.optim import Adam
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from auto_LiRPA import BoundedModule, BoundedTensor

from tqdm import tqdm
import pandas as pd
import numpy as np
from copy import deepcopy

C:\Users\lpott\anaconda3\envs\survival\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\lpott\anaconda3\envs\survival\lib\site-packages\torch\utils\cpp_extension.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import packaging  # type: ignore[attr-defined]


In [2]:
class ARGS(object):
    def __init__(self):
        pass

In [3]:
args = ARGS()
args.verify=False
args.device="cpu"

args.seed = 123

args.eps=0.5
args.norm=np.inf
args.bound_type = "CROWN-IBP"
args.num_epochs=10
args.lr = 1e-3
args.batch_size= 512
args.scheduler_name = "SmoothedScheduler"
args.scheduler_opts = "start=5,length=20"
args.hidden_dims = [15,15]
args.save_model = ""
args.dataset = "Dialysis"


In [4]:
# GOOD DATASETS
# 1. TRACE
# 2. divorce 
# 3. Dialysis
dataset_train,dataset_test = load_datasets(args.dataset,test_size=0.2)

In [5]:
input_dims = dataset_train.tensors[0].shape[1]
output_dim = 1

In [6]:
dataloader_train = DataLoader(dataset_train,batch_size=args.batch_size,shuffle=True)
dataloader_test = DataLoader(dataset_test,batch_size=args.batch_size,shuffle=False)

dataloader_train.mean = dataloader_test.mean = dataset_train.mean
dataloader_train.std = dataloader_test.std = dataset_train.std


dataset_train.tensors[0].shape

torch.Size([5444, 74])

In [7]:
import torch.nn as nn

In [8]:
X_train,T_train,E_train = dataloader_train.dataset.tensors

In [9]:
class RankingWrapper(nn.Module):
    def __init__(self,model):
        super(RankingWrapper,self).__init__()
        self.model = model

    def forward(self,x,t,e):
        R = self.model.failure_cdf(x, t)

        one_vector = torch.ones_like(t)
        
        Rii = 1.0 - torch.exp(-self.model(x)*t)
        # print(Rii.shape)

        #
        R_ = one_vector @ torch.transpose(Rii,1,0) #torch.reshape(torch.diag(R),(1,-1)) #.view(1, -1)
        # R_{ij} = r_{i}{T_{j}}
        #
        L = R_ - R
        # # R_{ij} = r_{j}(T_{j}) - r_{i}(T_{j})
        
        G = torch.transpose(L,1,0)
        # #
        T =  1.0 - torch.heaviside(t - torch.transpose(t,1,0),torch.tensor([1.0]))
        # # T_{ij}=1 if t_i < t_j  and T_{ij}=0 if t_i >= t_j
        
        T = T * e
        #  only remains T_{ij}=1 when event occured for subject i
        #
        sigma1 = 1.0
        #
        pairwise_ranking_loss = T * torch.exp(-G / sigma1)

        
        # pairwise_ranking_loss.mean(axis=1, keepdim=True)
        return torch.mean(pairwise_ranking_loss, axis=1,keepdims=True)#.view(-1, 1)

In [10]:
# remain = (T_train < T_train.T) * E_train

In [11]:
Xb = X_train[:25]
Tb = T_train[:25]
Eb = E_train[:25]

In [12]:
clf_robust = Exponential_Model(input_dim=input_dims,hidden_layers=args.hidden_dims)
clf_fragile = Exponential_Model(input_dim=input_dims,hidden_layers=args.hidden_dims)
clf_fragile.load_state_dict(deepcopy(clf_robust.state_dict()))


# # model = BoundedModule(clf, X_train)
model_robust_wrap = BoundedModule(RankingWrapper(clf_robust),(Xb,Tb,Eb))
model_fragile_wrap = BoundedModule(RankingWrapper(clf_fragile),(Xb,Tb,Eb))

C:\Users\lpott\AppData\Local\Temp\ipykernel_34104\3218600458.py:23: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  T =  1.0 - torch.heaviside(t - torch.transpose(t,1,0),torch.tensor([1.0]))


In [13]:
model_robust_wrap(Xb,Tb,Eb)

tensor([[0.0000],
        [0.5605],
        [0.0000],
        [0.8953],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.6611],
        [0.0000],
        [0.5427],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.2000],
        [0.0000]], grad_fn=<MeanBackward1>)

In [14]:
from auto_LiRPA.perturbations import *

epsilon = 1e-4
# Define perturbation. Here we add Linf perturbation to input data.
ptb = PerturbationLpNorm(norm=np.inf, eps=torch.Tensor([epsilon]))
# Make the input a BoundedTensor with the pre-defined perturbation.
my_input = BoundedTensor(torch.Tensor(Xb), ptb)
# Regular forward propagation using BoundedTensor works as usual.
# prediction = model(my_input,torch.rand_like(T_train),torch.rand_like(E_train))
# Compute LiRPA bounds using CROWN
lb, ub = model_robust_wrap.compute_bounds(x=(my_input, Tb, Eb), IBP=False, method="backward")

RuntimeError: number of dims don't match in permute